
# Strands Agent Graphを使用したマルチエージェントシステムの構築
マルチエージェントシステムは、複数の専門化されたAIエージェントが協調して複雑な問題を解決するために連携します。各エージェントは特定の能力と役割を持ち、明示的な通信経路を通じて接続されています。

このラボでは、Strands Agent SDKを使用してマルチエージェントシステムを構築する方法を学びます。基本的な概念から高度な実装まで進み、さまざまなトポロジーと実世界のアプリケーションを探求します。

**学習目標:**
このノートブックを終えるまでに、以下ができるようになります:
- エージェントグラフの3つの中核コンポーネント(ノード、エッジ、条件)を理解する
- 特定のエージェント間でターゲットメッセージを送信する
- マルチエージェントネットワークを監視および制御する
- 実世界のシナリオに対応した専門的なエージェントシステムを設計する

## 前提条件

- Python 3.10以上
- Amazon BedrockでAnthropic Claude 3.7が有効になっているAWSアカウント
- Amazon Bedrockを使用する権限を持つIAMロール
- AIエージェントとプロンプトエンジニアリングの基本的な理解

## セットアップとインストール

開始する前に、`strands-agents`と`strands-agents-tools`に必要なパッケージをインストールしましょう

In [ ]:
!pip install -r requirements.txt

### 必要なパッケージのインポート

次に必要なパッケージをインポートします

In [ ]:
from strands import Agent

## Agent Graphコンポーネントの理解
エージェントグラフは、協調的な協力を通じて複雑な問題を解決するために設計された、相互接続されたAIエージェントの構造化されたネットワークです。各エージェントは特定の能力を持つ専門化されたノードを表し、エージェント間の接続は明示的な通信経路を定義します。

構築を開始する前に、エージェントグラフの3つの主要なコンポーネントを理解しましょう:

### 1. ノード(エージェント)
各ノードは以下を持つAIエージェントを表します:
- **アイデンティティ**: グラフ内の一意の識別子
- **役割**: 専門化された機能または目的
- **システムプロンプト**: エージェントの動作を定義する指示
- **ツール**: エージェントが使用できる能力

### 2. エッジ(接続)
エッジは以下を持つ通信経路を定義します:
- **方向**: 一方向または双方向の情報フロー
- **条件**: エッジをトラバースすべきかどうかを決定するオプションの関数
- **依存関係**: ノード間の実行順序とデータフローを定義

### 3. GraphBuilder
GraphBuilderは、グラフを構築するためのシンプルなインターフェースを提供します:
- **add_node()**: エージェントまたはマルチエージェントシステムをノードとして追加
- **add_edge()**: ノード間の依存関係を作成
- **set_entry_point()**: 実行の開始ノードを定義
- **build()**: グラフインスタンスを検証して作成



### 基本的な処理

まず、2つの異なるエージェントによって処理される1つのタスクの簡単な例から始めましょう。これらのエージェントは定義された役割に応じて出力を提供します。ノードの実行順序と、Strands SDKでは必要に応じて1つのノードからのみ明示的に応答を取得できることに注目してください。アーキテクチャは以下のようになります:

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [ ]:
#agent_graph機能を持つエージェントを初期化
from strands.multiagent import GraphBuilder

# 専門化されたエージェントを作成
coordinator = Agent(name="coordinator", system_prompt="You are a research team leader coordinating specialists. Provide a short analysis, no need for follow ups")
analyst = Agent(name="data_analyst", system_prompt="You are a data analyst specializing in statistical analysis. Provide a short analysis, no need for follow ups")
domain_expert = Agent(name="domain_expert", system_prompt="You are a domain expert with deep subject knowledge. Provide a short analysis, no need for follow ups")

# グラフを構築
builder = GraphBuilder()

# ノードを追加
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# エッジ(依存関係)を追加
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# エントリーポイントを設定(オプション - 指定しない場合は自動検出)
builder.set_entry_point("team_lead")

# グラフを構築
graph = builder.build()

#新しく構築されたグラフでタスクを実行
result = graph("Analyze the impact of remote work on employee productivity.Provide a short analysis, no need for follow ups")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============ノードの実行順序:==========================")
print("============================================================")

# どのノードが実行されたか、どの順序で実行されたかを確認
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============グラフメトリクス:=================================")
print("============================================================")


# パフォーマンスメトリクスを取得
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# 特定のノードから結果を取得
print("\n")
print("=============エキスパートノードの結果のみ:======================")
print("============================================================")
print(result.results["expert"].result)

### 並列処理

次に、問題の2つの異なる側面を見て、リクエストを処理する2つのエージェントを持つトポロジーを作成し、提供された入力に基づいて要約とリスク計算を担当する最終エージェントに入力させましょう 
<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [ ]:
#agent_graph機能を持つエージェントを初期化
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 専門化されたエージェントを作成

financial_advisor = Agent(name="financial_advisor", system_prompt="You are a financial advisor focused on cost-benefit analysis, budget implications, and ROI calculations. Engage with other experts to build comprehensive financial perspectives.")
technical_architect = Agent(name="technical_architect", system_prompt="You are a technical architect who evaluates feasibility, implementation challenges, and technical risks. Collaborate with other experts to ensure technical viability.")
market_researcher = Agent(name="market_researcher", system_prompt="You are a market researcher who analyzes market conditions, user needs, and competitive landscape. Work with other experts to validate market opportunities.")
risk_analyst = Agent(name="risk_analyst", system_prompt="You are a risk analyst who identifies potential risks, mitigation strategies, and compliance issues. Collaborate with other experts to ensure comprehensive risk assessment.")


# グラフを構築
builder = GraphBuilder()

# ノードを追加
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# エッジ(依存関係)を追加
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# エントリーポイントを設定(オプション - 指定しない場合は自動検出)
builder.set_entry_point("finance_expert")

# グラフを構築
graph = builder.build()

print("============================================================")
print("============================================================")

#新しく構築されたグラフでタスクを実行
result = graph("Our company is considering launching a new AI-powered customer service platform. Initial investment is \$2M with projected 3-year ROI of 150%. What's your financial assessment?")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============ノードの実行順序:==========================")
print("============================================================")

# どのノードが実行されたか、どの順序で実行されたかを確認
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============グラフメトリクス:=================================")
print("============================================================")


# パフォーマンスメトリクスを取得
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")


# 特定のノードから結果を取得

print("Financial Advisor:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("Technical Expert:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("Market Researcher:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

### 条件付き分岐

リクエストを分類し、コード内で定義する条件に応じて、テクニカルエージェントまたはビジネスエージェントのいずれかにリクエストをルーティングするエージェントグラフを作成しましょう。

2つの異なるプロンプトに基づいて、このグラフのノード実行順序と実行されたノード数の違いに注目してください。

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>

In [ ]:
#agent_graph機能を持つエージェントを初期化
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 専門化されたエージェントを作成

classifier = Agent(name="classifier", system_prompt="You are an agent responsible for classification of the report request, return only Technical or Business clasification.")
technical_report = Agent(name="technical_expert", system_prompt="You are a technical expert htat focuses on providing short summary from technical perspective")
business_report = Agent(name="business_expert", system_prompt="You are a business expert that focuses on providing short summary from business perspective")

# グラフを構築
builder = GraphBuilder()

# ノードを追加
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# エッジ(依存関係)を追加
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# エントリーポイントを設定(オプション - 指定しない場合は自動検出)
builder.set_entry_point("classifier")

# グラフを構築
graph = builder.build()

print("============================================================")
print("============================================================")

#新しく構築されたグラフでタスクを実行
result = graph("Provide report on technical aspect of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============ノードの実行順序:==========================")
print("============================================================")

# どのノードが実行されたか、どの順序で実行されたかを確認
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============グラフメトリクス:=================================")
print("============================================================")


# パフォーマンスメトリクスを取得
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# 特定のノードから結果を取得

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

#新しく構築されたグラフでタスクを実行
result = graph("Provide report on business impact of working from home, outline things to consider and key risk factors")
print("\n")
print("============================================================")
print("============================================================")

print(f"Response: {result}")

print("=============ノードの実行順序:==========================")
print("============================================================")

# どのノードが実行されたか、どの順序で実行されたかを確認
for node in result.execution_order:
    print(f"Executed: {node.node_id}")

print("=============グラフメトリクス:=================================")
print("============================================================")


# パフォーマンスメトリクスを取得
print(f"Total nodes: {result.total_nodes}")
print(f"Completed nodes: {result.completed_nodes}")
print(f"Failed nodes: {result.failed_nodes}")
print(f"Execution time: {result.execution_time}ms")
print(f"Token usage: {result.accumulated_usage}")

# 特定のノードから結果を取得

print("Classifier:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

## 主な要点とベストプラクティス


### ベストプラクティス:

**非巡回性を考慮した設計:** グラフにサイクルがないことを確認する</p>
**意味のあるノードIDを使用:** ノードには説明的な名前を選択する</p>
**グラフ構造を検証:** ビルダーがサイクルをチェックし、エントリーポイントを検証する</p>
**ノードの失敗を処理:** 1つのノードの失敗が全体のワークフローにどのように影響するかを考慮する</p>
**条件付きエッジを使用:** 中間結果に基づく動的なワークフローに使用する</p>
**並列性を考慮:** 独立したブランチは同時に実行できる</p>
**マルチエージェントパターンをネスト:** 複雑なワークフローのためにGraph内でSwarmを使用する</p>
**マルチモーダル入力を活用:** 画像を含むリッチな入力にContentBlocksを使用する</p>

## まとめ

これで、Strands Agent Graphを使用したマルチエージェントシステムの構築の基礎をマスターしました!複雑な問題を解決するために協力する専門化されたAIエージェントの高度なネットワークを作成できるようになりました。

成功するマルチエージェントシステムの鍵は:
- トポロジーをユースケースに合わせる
- 明確なエージェントの役割と責任を定義する  
- 適切な通信パターンを確立する
- リソースとクリーンアップを効果的に管理する

ここから、研究、コンテンツ作成、意思決定、カスタマーサービスなど、実世界のアプリケーションのためのより高度なシステムを構築できます。